In [1]:
# imports:
from astropy.coordinates import SkyCoord, Galactic, ICRS, FK5
from astropy.table import Column, Table, join
from ligo.skymap.io import read_sky_map
from matplotlib import pyplot as plt
from astropy_healpix import HEALPix
import astropy.coordinates as coord
from astropy import units as u
from astropy.io import fits, ascii
import ligo.skymap.plot
from ligo.skymap.postprocess.contour import contour
import ligo.skymap.moc
import healpy as hp
import numpy as np
import scipy
import math
%matplotlib inline

In [2]:
# opening BBH LIGO data:
S190521g = '../data/S190521g/bayestar2.fits' 
bbh_data = Table.read(S190521g)

bbh_data
# multiorder maps: LIGO DCC gwtc2

PROB,DISTMU,DISTSIGMA,DISTNORM
1 / pix,Mpc,Mpc,1 / Mpc2
float64,float64,float64,float64
3.420666741585511e-21,-73.1221838420601,404.8957816151533,1.642233920921469e-05
3.420666741585511e-21,-73.1221838420601,404.8957816151533,1.642233920921469e-05
3.420666741585511e-21,-73.1221838420601,404.8957816151533,1.642233920921469e-05
3.420666741585511e-21,-73.1221838420601,404.8957816151533,1.642233920921469e-05
3.420666741585511e-21,-73.1221838420601,404.8957816151533,1.642233920921469e-05
3.420666741585511e-21,-73.1221838420601,404.8957816151533,1.642233920921469e-05
3.420666741585511e-21,-73.1221838420601,404.8957816151533,1.642233920921469e-05
3.420666741585511e-21,-73.1221838420601,404.8957816151533,1.642233920921469e-05


In [3]:
# adding an index row to prevent pixel scrambling:
index = Column(np.arange(len(bbh_data)), name='INDEX')
bbh_data.add_column(index, index=0)

bbh_data

INDEX,PROB,DISTMU,DISTSIGMA,DISTNORM
,1 / pix,Mpc,Mpc,1 / Mpc2
int64,float64,float64,float64,float64
0,3.420666741585511e-21,-73.1221838420601,404.8957816151533,1.642233920921469e-05
1,3.420666741585511e-21,-73.1221838420601,404.8957816151533,1.642233920921469e-05
2,3.420666741585511e-21,-73.1221838420601,404.8957816151533,1.642233920921469e-05
3,3.420666741585511e-21,-73.1221838420601,404.8957816151533,1.642233920921469e-05
4,3.420666741585511e-21,-73.1221838420601,404.8957816151533,1.642233920921469e-05
5,3.420666741585511e-21,-73.1221838420601,404.8957816151533,1.642233920921469e-05
6,3.420666741585511e-21,-73.1221838420601,404.8957816151533,1.642233920921469e-05
7,3.420666741585511e-21,-73.1221838420601,404.8957816151533,1.642233920921469e-05


In [4]:
# getting nside value:
nside = int((len(bbh_data))/12)**(1/2)

In [ ]:
bbh_ninety_area = isolate_90_area(bbh_data, order='PROB')
bbh_ninety_area